In [1]:
import pandas as pd
import numpy as np

## Changing the variable names

In [2]:
#reading the files
data=pd.read_excel('Survey_report.xlsx')
variables_desc=pd.read_excel('variable_description.xlsx')

In [3]:
data.shape

(1068, 49)

In [173]:
#Copying the data to another dataframe to rename the variables
data_1=data.drop(columns=['SurveyID','Building Name'])

In [174]:
#Changing the variable names
data_1.columns=variables_desc['Variable'].to_numpy()

In [175]:
#Adding survey_id and building_name columns to the data
data_1.insert(0, column='survey_id', value=data['SurveyID'])
data_1.insert(1, column='building_name', value=data['Building Name'])

In [176]:
#Export the data file with changed variable names
data_1.to_csv('data_with_new_variables.csv',index=False,)

## Reading the data after changing variables

In [5]:
data_1=pd.read_csv('data_with_new_variables.csv')
data_1.shape

(1068, 49)

In [6]:
#dropping other_view_description- It is having various text that need text analysis which might be considered in future
data_1=data_1.drop(columns=['other_view_description'])
#dropping work_building_name- It is same as the building_name
data_1=data_1.drop(columns=['work_building_name'])

In [7]:
#Changing the data type to category
for i in data_1.columns:
    if i!='survey_id':
        data_1[i]=data_1[i].astype('category')

In [8]:
#Replacing all nulls with No in views
view_type=['exterior_view','sky_view','Parking_view','other_buildings_view','trees_view','other_view']
data_2=data_1.copy()
for i in view_type:
    if 'No' not in data_2[i].unique():
        data_2[i]=data_2[i].cat.add_categories('No')
    data_2[i]=data_2[i].fillna('No')

In [10]:
#Replacing all nulls with No in controls
control_type=['blinds_control', 'overhead_lighting_control', 'task_light_control',
       'temperature_control', 'air_quality_control', 'other_control',
       'no_control']
data_3=data_2.copy()
for i in control_type:
    if 'No' not in data_3[i].unique():
        data_3[i]=data_3[i].cat.add_categories('No')
    data_3[i]=data_3[i].fillna('No') 
data_3.shape

(1068, 47)

In [11]:
#removing rows with more than 50% columns are missing
data_3=data_3[data_3.apply(lambda x: x.count(), axis=1)>24]
data_3.shape

(1058, 47)

In [317]:
# Replacing null values in personal_workspace,office_spending_space with other
data_3['personal_workspace']=data_3['personal_workspace'].fillna('Other')
data_3['office_spending_space']=data_3['office_spending_space'].fillna('Other')

In [318]:
#Replacing missing values in window_distance,building_floor with mode based on building_name
data_3.window_distance = data_3.groupby('building_name')['window_distance'].transform(lambda x: x.fillna(x.mode()[0]))
data_3.building_floor = data_3.groupby('building_name')['building_floor'].transform(lambda x: x.fillna(x.mode()[0]))

In [319]:
data_3.isna().sum()

survey_id                            0
building_name                        0
office_spending_space                0
personal_workspace                   0
in_out_office                       28
window_distance                      0
exterior_view                        0
sky_view                             0
Parking_view                         0
other_buildings_view                 0
trees_view                           0
other_view                           0
exterior_window_direction           16
building_floor                       0
workspace_rating                    24
thermal_condition                   37
cool_rating                         13
warm_rating                         32
temperature_alter_ability           13
temperature_satisfaction           229
clothing_layers_need                 7
outside_brake_need                  12
temperature_fluctuations_day        16
temperature_fluctuations_season     30
fan_usage                           13
leave_workspace          

In [320]:
data_4=data_3.copy()


In [321]:
data_4['personal_workspace'].unique()

[Open Office: workspace in open office with no ..., Enclosed Office: private, Enclosed Office: shared with other people, Open Office: cubicles with low partitions (low..., Open Office: cubicles with high partitions (5 ..., Other]
Categories (6, object): [Open Office: workspace in open office with no ..., Enclosed Office: private, Enclosed Office: shared with other people, Open Office: cubicles with low partitions (low..., Open Office: cubicles with high partitions (5 ..., Other]

In [322]:
replace_workspace={'Open Office: workspace in open office with no partitions':'Open Office_1',
                   'Open Office: cubicles with low partitions (lower than 5 feet)':'Open Office_2',
                  'Open Office: cubicles with high partitions (5 feet or higher)':'Open Office_3',
                  'Enclosed Office: private':'Enclosed Office_1',
                  'Enclosed Office: shared with other people':'Enclosed Office_2',
                  'Other':'Other'}
#replace_office_spending_space={'Personal workspace':1,'Outside':2,'Large conference room':3,
#                               'Small conference room':4,'Break room':5,'Other':6}

In [323]:
data_4=data_4.replace(replace_workspace)
#data_4=data_4.replace(replace_office_spending_space)

In [324]:
# for i in data_4.columns:
#     print(i, data_4[i].unique())
data_4.columns

Index(['survey_id', 'building_name', 'office_spending_space',
       'personal_workspace', 'in_out_office', 'window_distance',
       'exterior_view', 'sky_view', 'Parking_view', 'other_buildings_view',
       'trees_view', 'other_view', 'exterior_window_direction',
       'building_floor', 'workspace_rating', 'thermal_condition',
       'cool_rating', 'warm_rating', 'temperature_alter_ability',
       'temperature_satisfaction', 'clothing_layers_need',
       'outside_brake_need', 'temperature_fluctuations_day',
       'temperature_fluctuations_season', 'fan_usage', 'leave_workspace',
       'temperature_effect_productivity', 'afternoon_bright',
       'computer_screen_glare', 'evening_dark', 'task_light_usage',
       'unnecessary_lighting', 'lighting_effect_productivity',
       'blinds_control', 'overhead_lighting_control', 'task_light_control',
       'temperature_control', 'air_quality_control', 'other_control',
       'no_control', 'hours_spend', 'work_time', 'work_type',
      

In [325]:
lighting_columns=['afternoon_bright',
       'computer_screen_glare', 'evening_dark', 'task_light_usage',
       'unnecessary_lighting', 'lighting_effect_productivity']
data_5=data_4.drop(columns=lighting_columns)

In [326]:
data_5.shape

(1058, 41)

In [327]:
data_5.isna().sum()

survey_id                            0
building_name                        0
office_spending_space                0
personal_workspace                   0
in_out_office                       28
window_distance                      0
exterior_view                        0
sky_view                             0
Parking_view                         0
other_buildings_view                 0
trees_view                           0
other_view                           0
exterior_window_direction           16
building_floor                       0
workspace_rating                    24
thermal_condition                   37
cool_rating                         13
warm_rating                         32
temperature_alter_ability           13
temperature_satisfaction           229
clothing_layers_need                 7
outside_brake_need                  12
temperature_fluctuations_day        16
temperature_fluctuations_season     30
fan_usage                           13
leave_workspace          

In [328]:
#Changing the data type to category
for i in data_5.columns:
    if i!='survey_id':
        data_5[i]=data_5[i].astype('category')

In [329]:
data_5.to_csv('data_5.csv',index=False)

# Replacing values of the few variables for analysis purpose

In [281]:
data_6=data_5.copy()

In [282]:
replace_frequent={'Never':1, 'Rarely':2, 'Sometimes':3,'Most of the time':4, 'Always':5}
data_6=data_6.replace(replace_frequent)

In [283]:
replace_temp_effect={'3 = Neutral':3,'1 = Greatly decreses my productivity':1, '5 = Greatly increases my productivity':5}
data_6=data_6.replace(replace_temp_effect)

In [284]:
replace_window_distance={'0 - 5 feet':1,'6 - 10 feet':2,'11 - 15 feet':3,'16 feet or more':4}
data_6=data_6.replace(replace_window_distance)

In [285]:
replace_hours_spend={'Less than 10 hours':1,'10 - 20 hours':2,'21 - 40 hours':3,'More than 40 hours':4}
data_6=data_6.replace(replace_hours_spend)

In [286]:
replace_tenure={'Less than 1 year':1,'1 - 5 years':2,'5+ years':3}
data_6=data_6.replace(replace_tenure)

In [291]:
replace_age={'20 or below':1,'21 - 30':2,'31 - 40':3,'41 - 50':4,'51 or above':5,'I prefer not to answer':np.nan}
data_6=data_6.replace(replace_age)

In [295]:
data_6['building_floor'].unique()

[1 to 4, 5 to 9, 10 or above]
Categories (3, object): [1 to 4, 5 to 9, 10 or above]

In [296]:
replace_floor={'1 to 4':1,'5 to 9':2,'10 or above':3}
data_6=data_6.replace(replace_floor)

In [293]:
#data_6.to_csv('dataset_6.csv',index=False)

In [294]:
data_6.columns

Index(['survey_id', 'building_name', 'office_spending_space',
       'personal_workspace', 'in_out_office', 'window_distance',
       'exterior_view', 'sky_view', 'Parking_view', 'other_buildings_view',
       'trees_view', 'other_view', 'exterior_window_direction',
       'building_floor', 'workspace_rating', 'thermal_condition',
       'cool_rating', 'warm_rating', 'temperature_alter_ability',
       'temperature_satisfaction', 'clothing_layers_need',
       'outside_brake_need', 'temperature_fluctuations_day',
       'temperature_fluctuations_season', 'fan_usage', 'leave_workspace',
       'temperature_effect_productivity', 'blinds_control',
       'overhead_lighting_control', 'task_light_control',
       'temperature_control', 'air_quality_control', 'other_control',
       'no_control', 'hours_spend', 'work_time', 'work_type',
       'work_building_tenure', 'workspace_tenure', 'age_range', 'gender'],
      dtype='object')

In [297]:
data_7=data_6.copy()

In [301]:
Categorical_columns=['building_name','office_spending_space','personal_workspace', 'in_out_office',
                    'exterior_view', 'sky_view', 'Parking_view', 'other_buildings_view',
                    'trees_view', 'other_view','exterior_window_direction','blinds_control',
                    'overhead_lighting_control', 'task_light_control',
                    'temperature_control', 'air_quality_control', 'other_control',
                    'no_control','work_time', 'work_type','gender']

# Algorithms

We can use both the regression and classification algorithms for this to understand to get the answers for our problems

Regression analysis will given an out that will help to say which variables or effecting positively which have a negative effect on the satisfaction of the occupant

Classification will help to get the variables that are highly important or in other words factors that have more effect in estimating the occupant comfort

Regression algorithms: Multiple linear regression (MLR), Polynomial regression
Classification algorithms: Logistic regression or Multinomial Logistic Regression, Naive Bayes classifier, Randon Forest,
                           k-nearest neighbor, Neural networks, Boosting algorithms(AdaBoost,Gradient Tree Boosting, XGBoost)
                       